En el siguiente ntoebook se hará un nuevo preprocesado con tal objetivo de extraer un dataset que nos permita mejorar la precisión del modelo. Las decisiones que se tomen en este dataset derán en base a las últimas observaciones.

Las estadísticas proporcionadas por cada equipo son:
* FGM: Tiros de campo anotados
* FGA: TIros de campo intentados
* FGM3: Triples Anotados
* FGA3: Triples intentados
* FTM: Tiros libres anotados
* FTA: Tiros libres intentados
* OR: Rebites ofensivos
* DR: Rebotes defensivos
* Ast: Asistencias
* TO: Perdidas de balón
* Stl: Robos de balón
* Blk: Bloqueos
* PF: Faltas personales

In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
df.head(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


# Creación de nuevas variables

### Asist-To-Turnover Ratio

In [138]:
df['WA2TR'] = df['WAst'] / df['WTO']
df['LA2TR'] = df['LAst'] / df['LTO']
df['WA2TR-LA2TR'] = df['WA2TR'] - df['LA2TR']


## Convert Rebounds to a Ratio value
Convertimos los rebotes ofensivos al ratio de rebotes globales.

In [139]:
df['WOffR'] = df['WOR'] / (df['WOR'] + df['LDR'])
df['LOffR'] = df['LOR'] / (df['LOR'] + df['WDR'])
df['WOffR-LOffR'] = df['WOffR'] - df['LOffR']

###
df['WAllR'] = (df['WOR']+df['WDR']) / (df['WOR']+df['WDR']+df['LOR']+df['LDR'])
df['LAllR'] = 1 - df['WAllR']
df['WAllR-LAllR'] = df['WAllR'] - df['LAllR']



## Field Goal Percentage

In [140]:
df['WFGP'] = (df['WFGM']+df['WFGM3']) / (df['WFGA']+df['WFGA3'])
df['LFGP'] = (df['LFGM']+df['LFGM3']) / (df['LFGA']+df['LFGA3'])

## Ratio de Faltas Personales

In [141]:
df['WPFR'] = df['WPF'] / (df['WPF']+df['LPF'])
df['LPFR'] = df['LPF'] / (df['WPF']+df['LPF'])

## Removing already used columns

In [142]:
used_columns = ['LFGA', 'LFGA3', 'WFGA', 'WFGA3', 'WFGM', 'WFGM3', 'LFGM3',
                'LFGM', 'WOR','WDR', 'LOR', 'LDR', 'WAst', 'LAst', 'WTO', 'LTO']

uninformative_columns = ['WFTM', 'WFTA', 'LFTM', 'LFTA', 'WStl', 'LStl', 'WBlk', 'LBlk', 'NumOT',
                         'WAllR-LAllR', 'WA2TR-LA2TR', 'WA2TR-LA2TR', 'WA2TR-LA2TR', 'WAllR-LAllR',
                         'WOffR-LOffR', 'LScore', 'WScore']

drop_columns = used_columns + uninformative_columns

In [143]:
df.drop(columns=drop_columns, inplace=True)


In [144]:
df.drop(columns=['WLoc']).median()

Season     2014.000000
DayNum       75.000000
WTeamID    1287.000000
LTeamID    1281.000000
WPF          17.000000
LPF          19.000000
WA2TR         1.166667
LA2TR         0.818182
WOffR         0.321429
LOffR         0.285714
WAllR         0.527778
LAllR         0.472222
WFGP          0.450000
LFGP          0.377551
WPFR          0.470588
LPFR          0.529412
dtype: float64

## Convert dataset into something trainable

In [145]:
df["Home"] = np.where(df["WLoc"] == "H", df["WTeamID"], df["LTeamID"])
df["Away"] = np.where(df["WLoc"] != "H", df["WTeamID"], df["LTeamID"])
df["Result"] = np.where(df["WLoc"] == "H", 1, 0) # 1 Si gana el de casa, 0 si gana el de fuera

df['HPF'] = np.where(df["WLoc"] == "H", df['WPF'], df['LPF'])
df['APF'] = np.where(df["WLoc"] != "H", df['WPF'], df['LPF'])

df['HA2TR'] = np.where(df["WLoc"] == "H", df['WA2TR'], df['LA2TR'])
df['AA2TR'] = np.where(df["WLoc"] != "H", df['WA2TR'], df['LA2TR'])

df['HOffR'] = np.where(df["WLoc"] == "H", df['WOffR'], df['LOffR'])
df['AOffR'] = np.where(df["WLoc"] != "H", df['WOffR'], df['LOffR'])

df['HAllR'] = np.where(df["WLoc"] == "H", df['WAllR'], df['LAllR'])
df['AAllR'] = np.where(df["WLoc"] != "H", df['WAllR'], df['LAllR'])

df['HFGP'] = np.where(df["WLoc"] == "H", df['WFGP'], df['LFGP'])
df['AFGP'] = np.where(df["WLoc"] != "H", df['WFGP'], df['LFGP'])

df['HPFR'] = np.where(df["WLoc"] == "H", df['WPFR'], df['LPFR'])
df['APFR'] = np.where(df["WLoc"] != "H", df['WPFR'], df['LPFR'])


df.drop(columns=['WTeamID', 'LTeamID', 'WPF', 'LPF', 'WA2TR', 'LA2TR', 'WOffR', 'LOffR', 
                 'LAllR', 'WAllR', 'WFGP', 'LFGP', 'WPFR', 'LPFR', 'WLoc'], inplace=True)


In [146]:
df

,Season,DayNum,Home,Away,Result,HPF,APF,HA2TR,AA2TR,HOffR,AOffR,HAllR,AAllR,HFGP,AFGP,HPFR,APFR
0,2003,10,1328,1104,0,20,22,0.444444,0.565217,0.294118,0.388889,0.457143,0.542857,0.380952,0.416667,0.476190,0.523810
1,2003,10,1393,1272,0,16,18,0.583333,1.230769,0.416667,0.375000,0.511364,0.488636,0.329670,0.414634,0.470588,0.529412
2,2003,11,1437,1266,0,23,25,0.750000,1.500000,0.543860,0.435897,0.552083,0.447917,0.252525,0.421053,0.479167,0.520833
3,2003,11,1457,1296,0,23,18,0.473684,0.916667,0.472222,0.230769,0.596774,0.403226,0.338028,0.446809,0.560976,0.439024
4,2003,11,1208,1400,0,14,20,1.200000,0.857143,0.488372,0.531250,0.480000,0.520000,0.384615,0.480000,0.411765,0.588235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113120,2024,128,1177,1437,0,9,11,2.800000,1.700000,0.194444,0.100000,0.515152,0.484848,0.352273,0.379310,0.450000,0.550000
113121,2024,128,1323,1448,0,16,19,0.700000,1.428571,0.290323,0.406250,0.444444,0.555556,0.277778,0.426667,0.457143,0.542857
113122,2024,128,1349,1455,0,13,13,2.750000,3.666667,0.333333,0.393939,0.483333,0.516667,0.520548,0.505495,0.500000,0.500000
113123,2024,128,1139,1462,0,15,16,0.888889,1.357143,0.176471,0.235294,0.470588,0.529412,0.409091,0.426966,0.483871,0.516129


In [147]:
list(df.median())

[2014.0,
 75.0,
 1284.0,
 1284.0,
 1.0,
 17.0,
 19.0,
 1.0769230769230769,
 0.8888888888888888,
 0.30952380952380953,
 0.29411764705882354,
 0.5079365079365079,
 0.4920634920634921,
 0.41975308641975306,
 0.40625,
 0.4827586206896552,
 0.5172413793103449]

In [168]:

home_columns = ['HPF', 'HA2TR', 'HOffR', 'HAllR', 'HFGP', 'HPFR']
away_columns = ['APF', 'AA2TR', 'AOffR', 'AAllR', 'AFGP', 'APFR']
new_column_names = ['PF', 'A2TR', 'OffR', 'AllR', 'FGP', 'PFR']

dict_away = {x:v for x,v in zip(away_columns, new_column_names)}
dict_home = {x:v for x,v in zip(home_columns, new_column_names)}



final_df_columns = ['Season', 'DayNum', 'Home', 'Away'] + home_columns + away_columns
final_df = pd.DataFrame(columns=final_df_columns)

for idx, row in df.iterrows():
    home_home_last = df[(df['Season'] == row['Season']) & (df['DayNum']<row['DayNum']) & (df['Home'] == row['Home'])]
    home_away_last = df[(df['Season'] == row['Season']) & (df['DayNum']<row['DayNum']) & (df['Away'] == row['Home'])]
    
    away_home_last = df[(df['Season'] == row['Season']) & (df['DayNum']<row['DayNum']) & (df['Home'] == row['Away'])]
    away_away_last = df[(df['Season'] == row['Season']) & (df['DayNum']<row['DayNum']) & (df['Away'] == row['Away'])]
    
    home_home_last = home_home_last[home_columns]
    home_away_last = home_away_last[away_columns]

    away_home_last = away_home_last[home_columns]
    away_away_last = away_away_last[away_columns]


    home_home_last = home_home_last.rename(columns=dict_home)
    home_away_last = home_home_last.rename(columns=dict_away)
    away_home_last = away_home_last.rename(columns=dict_home)
    away_away_last = away_away_last.rename(columns=dict_away)
    
    away_last = pd.concat([away_away_last, away_home_last])
    home_last = pd.concat([home_away_last, home_home_last])

    home = list(home_last.median())
    away = list(away_last.median())

    final_df.loc[len(final_df)] = list(row[['Season', 'DayNum', 'Home', 'Away']].values) + home + away

    print(f"{idx}/{len(df)}")


0/113125
1/113125
2/113125
3/113125
4/113125
5/113125
6/113125
7/113125
8/113125
9/113125
10/113125
11/113125
12/113125
13/113125
14/113125
15/113125
16/113125
17/113125
18/113125
19/113125
20/113125
21/113125
22/113125
23/113125
24/113125
25/113125
26/113125
27/113125
28/113125
29/113125
30/113125
31/113125
32/113125
33/113125
34/113125
35/113125
36/113125
37/113125
38/113125
39/113125
40/113125
41/113125
42/113125
43/113125
44/113125
45/113125
46/113125
47/113125
48/113125
49/113125
50/113125
51/113125
52/113125
53/113125
54/113125
55/113125
56/113125
57/113125
58/113125
59/113125
60/113125
61/113125
62/113125
63/113125
64/113125
65/113125
66/113125
67/113125
68/113125
69/113125
70/113125
71/113125
72/113125
73/113125
74/113125
75/113125
76/113125
77/113125
78/113125
79/113125
80/113125
81/113125
82/113125
83/113125
84/113125
85/113125
86/113125
87/113125
88/113125
89/113125
90/113125
91/113125
92/113125
93/113125
94/113125
95/113125
96/113125
97/113125
98/113125
99/113125
100/113125

In [157]:
len(list(row[['Season', 'DayNum', 'Home', 'Away']].values) + home + away)

22

In [166]:
away_last

,PF,A2TR,OffR,AllR,FGP,PFR,HPF,HA2TR,HOffR,HAllR,HFGP,HPFR


In [158]:
len(final_df_columns)

16

In [153]:
pd.concat()

{'HPF': 'APF',
 'HA2TR': 'AA2TR',
 'HOffR': 'AOffR',
 'HAllR': 'AAllR',
 'HFGP': 'AFGP',
 'HPFR': 'APFR'}

In [151]:
home_home_last.rename(columns=)

TypeError: 'list' object is not callable